In [1]:
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers
from kerastuner.tuners import RandomSearch, BayesianOptimization

In [2]:
df_train = pd.read_csv('/home/ubuntu/cs230-project/data_sub_PCA/df_train_PCA_dict_df_metab.csv')
print (df_train)

     Depressed at Time  df_metab_0  df_metab_1  df_metab_2  df_metab_3  \
0                 True   45.789637   14.994401    2.792549   -7.301174   
1                False   21.305736  -12.739211    2.414824  -22.494078   
2                False  -11.160583    0.500590    1.948961  -18.053438   
3                False   -3.505071    3.023377  -14.520562    9.177950   
4                False    2.370956   14.234072    6.691360    4.755687   
..                 ...         ...         ...         ...         ...   
123              False    0.125152   -2.485553   -3.342023  -20.668625   
124              False  -16.114327  -22.152604  -23.709256  -18.545960   
125              False   36.381486    6.960738   11.266882    9.134153   
126              False   -2.210158   12.276662  -22.779149   -9.248137   
127              False   -1.710068   17.256411   11.794209    3.967151   

     df_metab_4  df_metab_5  df_metab_6  df_metab_7  df_metab_8  ...  \
0      2.979374   -0.913556  -10.229722

In [6]:
y_train = df_train.iloc[:,0]
X_train = df_train.iloc[:,1:]

In [10]:
y_train = pd.get_dummies(y_train)

In [11]:
print (y_train.shape)
print (X_train.shape)

(128, 2)
(128, 88)


In [23]:
def build_model(hp):
    model = keras.Sequential()
    model.add(layers.Dropout(hp.Float('dropout', 0, 0.5, step = 0.1)))
    for i in range(hp.Int('num_layers', 1, 4)): #16)):
        model.add(layers.Dense(units = hp.Int('units_'+str(i), 20, 100, step = 10),
        #model.add(layers.Dense(units = hp.Int('units_'+str(i), 25, 1000, step = 25),
                               activation = hp.Choice('act_'+str(i), ['relu', 'tanh'])))
    model.add(layers.Dense(2, activation = 'softmax')) #2 for binary classification
    
    model.compile(
        optimizer = keras.optimizers.RMSprop(
            learning_rate = hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4]),
            rho = hp.Float('rho', 0.5, 0.99, step = 0.05),
            momentum = hp.Float('momentum', 0, 0.5, step = 0.05)),
        loss = 'binary_crossentropy',
        metrics = ['accuracy'])
    return model

In [26]:
#tuner = RandomSearch(
tuner = BayesianOptimization(
    build_model,
    objective = 'acc',
    max_trials = 5,
    executions_per_trial = 3,
    directory = 'Metabolomics')

INFO:tensorflow:Reloading Oracle from existing project Metabolomics/untitled_project/oracle.json


In [27]:
tuner.search(X_train.values, y_train.values,
             epochs=5)

Train on 128 samples
Epoch 1/5
128/128 [==============================] - ETA: 0s - loss: 1.1779 - acc: 0.250 - 0s 2ms/sample - loss: 0.6644 - acc: 0.6484
Epoch 2/5
128/128 [==============================] - ETA: 0s - loss: 0.3614 - acc: 0.843 - 0s 223us/sample - loss: 0.2994 - acc: 0.9141
Epoch 3/5
128/128 [==============================] - ETA: 0s - loss: 0.1537 - acc: 0.968 - 0s 180us/sample - loss: 0.2108 - acc: 0.9219
Epoch 4/5
128/128 [==============================] - ETA: 0s - loss: 0.1467 - acc: 0.968 - 0s 209us/sample - loss: 0.1563 - acc: 0.9453
Epoch 5/5
128/128 [==============================] - ETA: 0s - loss: 0.0843 - acc: 0.968 - 0s 464us/sample - loss: 0.1063 - acc: 0.9531
Train on 128 samples
Epoch 1/5
128/128 [==============================] - ETA: 0s - loss: 0.7448 - acc: 0.500 - 0s 963us/sample - loss: 0.4665 - acc: 0.8047
Epoch 2/5
128/128 [==============================] - ETA: 0s - loss: 0.3436 - acc: 0.906 - 0s 119us/sample - loss: 0.2886 - acc: 0.9141
Epoch 3/

Train on 128 samples
Epoch 1/5
128/128 [==============================] - ETA: 0s - loss: 6.2871 - acc: 0.406 - 0s 1ms/sample - loss: 4.0127 - acc: 0.5781
Epoch 2/5
128/128 [==============================] - ETA: 0s - loss: 0.1528 - acc: 0.937 - 0s 163us/sample - loss: 1.2705 - acc: 0.8672
Epoch 3/5
128/128 [==============================] - ETA: 0s - loss: 0.3506 - acc: 0.968 - 0s 162us/sample - loss: 1.1116 - acc: 0.8984
Epoch 4/5
128/128 [==============================] - ETA: 0s - loss: 4.2687e-04 - acc: 1.000 - 0s 118us/sample - loss: 1.0002 - acc: 0.8984
Epoch 5/5
128/128 [==============================] - ETA: 0s - loss: 0.1283 - acc: 0.937 - 0s 167us/sample - loss: 0.6469 - acc: 0.9141
Train on 128 samples
Epoch 1/5
128/128 [==============================] - ETA: 0s - loss: 3.1587 - acc: 0.593 - 0s 816us/sample - loss: 2.2571 - acc: 0.7266
Epoch 2/5
128/128 [==============================] - ETA: 0s - loss: 0.6981 - acc: 0.906 - 0s 115us/sample - loss: 1.1926 - acc: 0.8672
Epoc

Train on 128 samples
Epoch 1/5
128/128 [==============================] - ETA: 0s - loss: 0.6323 - acc: 0.687 - 0s 1ms/sample - loss: 0.6057 - acc: 0.7109
Epoch 2/5
128/128 [==============================] - ETA: 0s - loss: 0.3634 - acc: 0.906 - 0s 166us/sample - loss: 0.3966 - acc: 0.8672
Epoch 3/5
128/128 [==============================] - ETA: 0s - loss: 0.3225 - acc: 0.937 - 0s 291us/sample - loss: 0.2909 - acc: 0.9375
Epoch 4/5
128/128 [==============================] - ETA: 0s - loss: 0.2705 - acc: 0.875 - 0s 116us/sample - loss: 0.2463 - acc: 0.9219
Epoch 5/5
128/128 [==============================] - ETA: 0s - loss: 0.1709 - acc: 0.937 - 0s 112us/sample - loss: 0.1859 - acc: 0.9375
Train on 128 samples
Epoch 1/5
128/128 [==============================] - ETA: 0s - loss: 0.6530 - acc: 0.656 - 0s 899us/sample - loss: 0.5647 - acc: 0.7188
Epoch 2/5
128/128 [==============================] - ETA: 0s - loss: 0.4971 - acc: 0.750 - 0s 113us/sample - loss: 0.4226 - acc: 0.8359
Epoch 3/

Train on 128 samples
Epoch 1/5
128/128 [==============================] - ETA: 0s - loss: 3.7259 - acc: 0.156 - 0s 2ms/sample - loss: 1.6352 - acc: 0.5469
Epoch 2/5
128/128 [==============================] - ETA: 0s - loss: 0.9109 - acc: 0.781 - 0s 189us/sample - loss: 0.5528 - acc: 0.8516
Epoch 3/5
128/128 [==============================] - ETA: 0s - loss: 0.5900 - acc: 0.812 - 0s 179us/sample - loss: 0.4779 - acc: 0.8750
Epoch 4/5
128/128 [==============================] - ETA: 0s - loss: 0.1763 - acc: 0.906 - 0s 390us/sample - loss: 0.3647 - acc: 0.8906
Epoch 5/5
128/128 [==============================] - ETA: 0s - loss: 0.2672 - acc: 0.875 - 0s 126us/sample - loss: 0.3129 - acc: 0.8828
Train on 128 samples
Epoch 1/5
128/128 [==============================] - ETA: 0s - loss: 2.0600 - acc: 0.406 - 0s 975us/sample - loss: 0.9942 - acc: 0.7422
Epoch 2/5
128/128 [==============================] - ETA: 0s - loss: 0.3055 - acc: 0.906 - 0s 127us/sample - loss: 0.5321 - acc: 0.8672
Epoch 3/

Train on 128 samples
Epoch 1/5
128/128 [==============================] - ETA: 0s - loss: 0.8724 - acc: 0.468 - 0s 1ms/sample - loss: 0.8501 - acc: 0.4844
Epoch 2/5
128/128 [==============================] - ETA: 0s - loss: 0.6028 - acc: 0.656 - 0s 172us/sample - loss: 0.7178 - acc: 0.5703
Epoch 3/5
128/128 [==============================] - ETA: 0s - loss: 0.8700 - acc: 0.437 - 0s 176us/sample - loss: 0.7052 - acc: 0.6016
Epoch 4/5
128/128 [==============================] - ETA: 0s - loss: 0.7234 - acc: 0.531 - 0s 116us/sample - loss: 0.6548 - acc: 0.5938
Epoch 5/5
128/128 [==============================] - ETA: 0s - loss: 0.6059 - acc: 0.687 - 0s 171us/sample - loss: 0.5834 - acc: 0.6953
Train on 128 samples
Epoch 1/5
128/128 [==============================] - ETA: 0s - loss: 0.8798 - acc: 0.375 - 0s 900us/sample - loss: 0.7742 - acc: 0.5156
Epoch 2/5
128/128 [==============================] - ETA: 0s - loss: 0.6992 - acc: 0.531 - 0s 119us/sample - loss: 0.7197 - acc: 0.5234
Epoch 3/

INFO:tensorflow:Oracle triggered exit


In [28]:
tuner.results_summary()

In [56]:
best_hp = tuner.get_best_hyperparameters(num_trials=1)[0]
print (best_hp.values)

{'dropout': 0.2, 'num_layers': 3, 'units_0': 40, 'act_0': 'tanh', 'learning_rate': 0.01, 'rho': 0.55, 'momentum': 0.5, 'units_1': 20, 'act_1': 'relu', 'units_2': 20, 'act_2': 'relu'}


In [30]:
best_model = tuner.get_best_models(num_models=1)[0]

In [57]:
print (best_model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dropout (Dropout)            multiple                  0         
_________________________________________________________________
dense (Dense)                multiple                  3560      
_________________________________________________________________
dense_1 (Dense)              multiple                  820       
_________________________________________________________________
dense_2 (Dense)              multiple                  420       
_________________________________________________________________
dense_3 (Dense)              multiple                  42        
Total params: 4,842
Trainable params: 4,842
Non-trainable params: 0
_________________________________________________________________
None


In [35]:
df_dev = pd.read_csv('/home/ubuntu/cs230-project/data_sub_PCA/df_dev_PCA_dict_df_metab.csv')
print (df_dev.shape)

(37, 89)


In [36]:
y_dev = df_dev.iloc[:,0]
y_dev = pd.get_dummies(y_dev)
X_dev = df_dev.iloc[:,1:]

In [37]:
print (y_dev.shape)
print (X_dev.shape)

(37, 2)
(37, 88)


In [44]:
results = best_model.evaluate(X_dev.values, y_dev.values)

37/37 [==============================] - ETA: 0s - loss: 0.8509 - acc: 0.843 - 0s 186us/sample - loss: 0.7360 - acc: 0.8649
test loss, test acc: [0.7359883179998252, 0.8648649]


In [45]:
print ("test loss, test acc:", results)

test loss, test acc: [0.7359883179998252, 0.8648649]
